In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [ ]:

import sys
sys.path.append('../utils')
import functions

In [ ]:
df = pd.read_csv('../data/local/raw/doesthedogdie.com.csv')
df.head()

In [ ]:
functions.show_basic_info(df)

Check if there are duplicate titles

In [ ]:
duplicates = df[df['title'].duplicated()]

duplicate_count = df['title'].duplicated().sum()

print(f"Number of duplicates in 'title': {duplicate_count}")

### Check the categories in 'title' column

In [ ]:
# Extract the unique values before the underscore in 'title'
unique_titles = df['title'].str.split('_').str[0].unique()

print(unique_titles)

Extract media type and remove it from title column

In [ ]:
df['media_type'] = df['title'].str.split('_').str[0]  # Extract media type
df['title'] = df['title'].str.split('_').str[1]       # Remove media type from title

df.head()

In [ ]:
print(df['title'].unique())